In [1]:
%load_ext autoreload
%load_ext autotime

time: 213 µs (started: 2024-10-08 13:42:53 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 612 µs (started: 2024-10-08 13:42:53 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
trace_path = "/p/lustre3/iopp/dftracer-traces/v1.0.5-develop/corona/cm1_1_48_20240926/cm1_1_48_20240926-COMPACT/*.pfw.gz"

time: 741 µs (started: 2024-10-08 13:42:53 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-deepspeed-dlio-node-16"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/deepspeed-dlio-node-16"
trace_path = "/p/lustre3/iopp/dftracer-traces/v1.0.5-develop/corona/megatron_deepspeed-1_16_8/megatron_deepspeed-1_16_8-COMPACT/*.pfw.gz"


time: 1.08 ms (started: 2024-10-08 13:42:53 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 12.8 s (started: 2024-10-08 13:42:53 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    threshold=cfg.threshold,
    view_types=cfg.view_types,
)

time: 9min 37s (started: 2024-10-08 13:43:06 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          501.06 seconds                                                                                │
│  I/O Time         181.40 seconds                                                                                │
│                   ├── Read - 1.79 seconds (0.99%)                                                               │
│                   ├── Write - 158.26 seconds (87.24%)                                                           │
│                   └── Metadata - 24.63 seconds (13.58%)                                                         │
│  I/O Operations   25,740,376 ops                                                                                │
│                   ├── Read - 1,293,952 ops (5.03%)                                                              │
│                   ├── Write - 653,917 ops (2.54%)                                                               │
│                   └── Metadata - 23,298,811 ops (90.51%)                                                        │
│  I/O Size         5.72 TiB                                                                                      │
│                   ├── Read - 7.52 GiB (0.13%)                                                                   │
│                   └── Write - 5.71 TiB (99.87%)                                                                 │
│  Read Requests    4 kiB-1 MiB - 1,293,426 ops                                                                   │
│                   ├── <4 kiB - 641,399 ops (49.59%)                                                             │
│                   ├── 4-16 kiB - 370,563 ops (28.65%)                                                           │
│                   ├── 16-64 kiB - 242,872 ops (18.78%)                                                          │
│                   ├── 64-256 kiB - 36,288 ops (2.81%)                                                           │
│                   └── 256 kiB-1 MiB - 2,304 ops (0.18%)                                                         │
│  Write Requests   4 kiB->64 MiB - 653,917 ops                                                                   │
│                   ├── <4 kiB - 532,768 ops (81.47%)                                                             │
│                   ├── 4-16 kiB - 129 ops (0.02%)                                                                │
│                   ├── 16-64 kiB - 9,984 ops (1.53%)                                                             │
│                   ├── 1-4 MiB - 297 ops (0.05%)                                                                 │
│                   ├── 16-64 MiB - 2,591 ops (0.40%)                                                             │
│                   └── >64 MiB - 108,148 ops (16.54%)                                                            │
│  Nodes            16 nodes                                                                                      │
│                   ├── 26265 - 181.40 s (100.00%) - 481.36 MiB/365.34 GiB R/W (0.01/6.24%) - 1,608,261 ops       │
│                   │   (6.25%)                                                                                   │
│                   ├── 7923 - 181.32 s (99.95%) - 481.35 MiB/365.34 GiB R/W (0.01/6.24%) - 1,608,274 ops         │
│                   │   (6.25%)                                                                                   │
│                   ├── 50452 - 180.36 s (99.43%) - 481.35 MiB/365.34 GiB R/W (0.01/6.24%) - 1,608,251 ops        │
│                   │   (6.25%)                                                                                   │
│                   ├── 31683 - 180.34 s (99.41%) - 481.

time: 8min 33s (started: 2024-10-08 13:52:44 -07:00)
